<a href="https://colab.research.google.com/github/abhishek2602/Deep-Learning-with-NLP/blob/master/Sentiment_Analysis_RNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Movie Review Data

In [0]:
import numpy as np

In [0]:
np.random.seed(42) # Set the seed

In [0]:
import pandas as pd

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = "https://drive.google.com/open?id=15skTiFzK7cuAyGD6jUTlOfYq3FLU7dy5"
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('labeledTrainData.tsv')  
df = pd.read_csv('labeledTrainData.tsv', header = 0, delimiter = '\t', quoting = 3)

In [0]:
# Spliting the Data into Training and Test Data
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'], df['sentiment'], test_size = 0.2,
    random_state = 42)

## Build the Tokenizer

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [0]:
top_words = 10000

In [0]:
t = Tokenizer(num_words = top_words)

In [0]:
t.fit_on_texts(X_train.tolist())

## Prepare Training and Test Data

In [0]:
# Get each word index for each of the word in the review
X_train = t.texts_to_sequences(X_train.tolist())
X_test = t.texts_to_sequences(X_test.tolist())

## Pad Sequence 

In [0]:
from tensorflow.python.keras.preprocessing import sequence

In [0]:
max_review_length = 300

In [0]:
X_train = sequence.pad_sequences(X_train, maxlen = max_review_length, padding = 'post')
X_test = sequence.pad_sequences(X_test, maxlen = max_review_length, padding = 'post')

## Build Embedding Matrix rom Pre-trained Word2Vec

In [0]:
import gensim

In [0]:
word2vec = gensim.models.Word2Vec.load('word2vec-movie-50')

In [19]:
embedding_vector_length = word2vec.wv.syn0.shape[1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [0]:
embedding_matrix = np.zeros((top_words + 1, embedding_vector_length))

In [0]:
for word, i in sorted(t.word_index.items(), key = lambda x: x[1]):
    if i > top_words:
        break
    if word in word2vec.wv.vocab:
        embedding_vector = word2vec.wv[word]
        embedding_matrix[i] = embedding_vector

## Build the Graph

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten, LSTM

In [0]:
model = Sequential()

In [24]:
model.add(Embedding(top_words + 1,
                    embedding_vector_length,
                    input_length = max_review_length,
                    weights = [embedding_matrix],
                    trainable = False))

Instructions for updating:
Colocations handled automatically by placer.


In [30]:
model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.add(Dense(1, activation = 'sigmoid'))

In [0]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Execute the graph

In [33]:
model.fit(X_train, y_train,
          epochs = 30,
          batch_size = 128,
          validation_data = (X_test, y_test),
          verbose = 1)

Train on 20000 samples, validate on 5000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
20000/20000 [==============================] - 202s 10ms/sample - loss: 0.6937 - acc: 0.5011 - val_loss: 0.6931 - val_acc: 0.5034
Epoch 2/30
20000/20000 [==============================] - 201s 10ms/sample - loss: 0.6933 - acc: 0.4996 - val_loss: 0.6931 - val_acc: 0.5038
Epoch 3/30
20000/20000 [==============================] - 202s 10ms/sample - loss: 0.6932 - acc: 0.5057 - val_loss: 0.6930 - val_acc: 0.5042
Epoch 4/30
20000/20000 [==============================] - 200s 10ms/sample - loss: 0.6932 - acc: 0.5023 - val_loss: 0.6931 - val_acc: 0.5022
Epoch 5/30
20000/20000 [==============================] - 201s 10ms/sample - loss: 0.6929 - acc: 0.5082 - val_loss: 0.6928 - val_acc: 0.5136
Epoch 6/30
20000/20000 [==============================] - 200s 10ms/sample - loss: 0.6923 - acc: 0.5161 - val_loss: 0.6926 - val_acc: 0.5168
Epoch 7/30
20000/20000 [==============================] -